In [ ]:
import numpy as np

#Bayesian classifier:
class BernoulliNaiveBayes:
  def __init__(self, dataset, labels):
    """
    labels : 1d array, incude index of classes, start at 0
    dataset: (number_sample, num_features)
    """
    self.N = dataset.shape[0]
    self.num_classes = int(np.amax(labels) + 1)
    self.prior_value = self.get_prior_value(labels)
    self.likelihood_value = self.get_likelihood_value(dataset, labels)
    
  def get_prior_value(self, labels):
    """
    return all prior probability of all classes
    """
    result = np.empty((self.num_classes, ))
    for c_index in np.arange(self.num_classes):
      result[c_index] = np.sum(labels == c_index, dtype=np.float32) / self.N; 
    return result

  def predict(self, x):
    """
    Predict a class for new sample
    x = (num_sample, num_features) 
    """
    assert (x.shape[1] == self.likelihood_value.shape[1])
    
    posterior_value = np.empty((x.shape[0], self.num_classes))
    for c_index in np.arange(self.num_classes):
      posterior_value[:, c_index] = \
              np.log(self.prior_value[c_index]) +  \
              np.sum(
                  np.log(
                      np.absolute(
                          np.equal(x, 0) - self.likelihood_value[c_index])), 
                  axis=1)
    return np.argmax(posterior_value, axis=1)              
  
  def evaluate(self, x, y):
    """
    return:
      miss_classify:  sum(predicted_class == y)
    error_function = Sum((1 - P(C|X) * P(X))
    """
    predict_index_classes = self.predict(x)
    return np.sum(np.equal(predict_index_classes, y), )

  def predict_proba(self, x):
    posterior_value = np.empty((x.shape[0], self.num_classes))
    for c_index in np.arange(self.num_classes):
      posterior_value[:, c_index] = \
              self.prior_value[c_index] *  \
              np.prod(
                      np.absolute(
                          np.equal(x, 0) - self.likelihood_value[c_index]), 
                  axis=1)
    return posterior_value / np.sum(posterior_value, axis=1)[:, None]

   
  def get_likelihood_value(self, dataset, labels):
    result = np.empty((self.num_classes, dataset.shape[1]))            #(num_classes, num_feature)
    for c_index in np.arange(self.num_classes):                       #For all classes
      #Number of sample have this feature == 1 
      #and have this label in dataset
      num_feature_labels = np.sum(
                (dataset == 1) & (labels == c_index)[:, None],  #For broadcast
                dtype=np.float32, axis=0)
      #Number of  lables in the dataset
      num_labels = np.sum(labels == c_index, dtype=np.float32)
      #Conditional probability: P(F/C)
      result[c_index, :] = (num_feature_labels + 1) / (num_labels + 2)
    return result